In [ ]:
import numpy as np

In [ ]:
# Cargar los datos VariableX.mat y VariableX_Y.mat
import scipy.io as sio
import numpy as np


# SOLO si estás en Colab
from google.colab import files
uploaded = files.upload()

# Cargar los archivos .mat
data_X = sio.loadmat('VariableX.mat')
data_XY = sio.loadmat('VariableX_Y.mat')


Saving VariableX.mat to VariableX (1).mat
Saving VariableX_Y.mat to VariableX_Y (1).mat


In [ ]:
print("Contenido de VariableX.mat:")
print(data_X.keys())

print("\nContenido de VariableX_Y.mat:")
print(data_XY.keys())


Contenido de VariableX.mat:
dict_keys(['__header__', '__version__', '__globals__', 'X'])

Contenido de VariableX_Y.mat:
dict_keys(['__header__', '__version__', '__globals__', 'XY'])


In [ ]:
# Extraer las variables reales
X = data_X['X']
XY = data_XY['XY'] #eSTO ES LO MISMO QUE REALIZACIONES

# Mostrar formas originales
print("Forma original X_real:", X.shape)
print("Forma original XY_real:", XY.shape)


Forma original X_real: (5000, 1)
Forma original XY_real: (10000, 2)


## 1. Cálculo de entropía

In [ ]:
def entropy_estimation(data):
    unique_values, counts = np.unique(data, return_counts=True)
    probabilities = counts / len(data)  # Probabilidades estimadas (frecuencias relativas)
    entropy = -np.sum(probabilities * np.log2(probabilities))  # Cálculo de entropía
    return entropy
H_X = entropy_estimation(X)


# Mostrar los resultados
print(f"La entropía estimada H(X) es: {H_X:.4f} bits")


La entropía estimada H(X) es: 1.7703 bits


## 2. Cálculo de entropía conjunta

In [ ]:
#Entropía conjunta

def calculo_entropia_conjunta(realizaciones):
    """
    Calcula el alfabeto de X e Y, las probabilidades conjuntas y las entropías de X y Y.

    Parámetros:
        realizaciones (array): Matriz de tamaño N x 2, donde cada columna contiene las realizaciones de una variable aleatoria.

    Retornos:
        alfabeto_x (array): Vector columna con los valores únicos de X, ordenado de menor a mayor.
        alfabeto_y (array): Vector columna con los valores únicos de Y, ordenado de menor a mayor.
        probabilidadConjunta (2D array): Matriz Mx x My con las probabilidades conjuntas de X e Y.
        entropiaX (float): Entropía de la variable X.
        entropiaY (float): Entropía de la variable Y.
        entropiaConjunta (float): Entropía conjunta de X y Y.
    """
    # Extraer las columnas de la matriz de realizaciones como las variables aleatorias X e Y
    X = realizaciones[:, 0]
    Y = realizaciones[:, 1]

    # Obtener los valores únicos (alfabeto) de X e Y, ordenados
    alfabeto_x = np.unique(X)
    alfabeto_y = np.unique(Y)

    # Determinar el tamaño de los alfabetos de X e Y
    Mx = len(alfabeto_x)
    My = len(alfabeto_y)

    # Inicializar una matriz de frecuencias conjuntas con ceros
    frecuenciaConjunta = np.zeros((Mx, My))

    # Contar las frecuencias conjuntas para cada par (valor_x, valor_y)
    for i, valor_x in enumerate(alfabeto_x):
        for j, valor_y in enumerate(alfabeto_y):
            frecuenciaConjunta[i, j] = np.sum((X == valor_x) & (Y == valor_y))

    # Calcular la matriz de probabilidades conjuntas normalizando por el número total de realizaciones
    probabilidadConjunta = frecuenciaConjunta / len(X)

    # Calcular las frecuencias marginales de X e Y sumando las probabilidades conjuntas
    pX = np.sum(probabilidadConjunta, axis=1)
    pY = np.sum(probabilidadConjunta, axis=0)

    # Evitar log2(0) sumando un pequeño valor eps
    eps = 1e-12

    # Calcular la entropía de X
    entropiaX = entropy_estimation(X)

    # Calcular la entropía de Y
    entropiaY = entropy_estimation(Y)

    # Calcular la entropía conjunta H(X, Y)
    entropiaConjunta = -np.sum(probabilidadConjunta * np.log2(probabilidadConjunta + eps))

    # Retornar los resultados: alfabetos, probabilidades conjuntas y entropías
    return alfabeto_x, alfabeto_y, probabilidadConjunta, entropiaX, entropiaY, entropiaConjunta

_,_,prob_conj, entr_X, entr_Y, entr_conj = calculo_entropia_conjunta(XY)
entr_conj


np.float64(3.584267401607793)

## 3. Cálculo de entropía condicional

In [ ]:
def calculo_entropia_condicional(realizaciones):
    """
    Calcula el alfabeto de X e Y, las probabilidades condicionales y las entropías condicionales de X dado Y y Y dado X.

    Parámetros:
        realizaciones (array): Matriz de tamaño N x 2, donde cada columna contiene las realizaciones de una variable aleatoria.

    Retornos:
        alfabeto_x (array): Valores únicos de X, ordenados.
        alfabeto_y (array): Valores únicos de Y, ordenados.
        probabilidad_X_dado_Y (2D array): Matriz Mx x My con P(X|Y).
        probabilidad_Y_dado_X (2D array): Matriz Mx x My con P(Y|X).
        entropia_X_dado_Y (float): H(X | Y).
        entropia_Y_dado_X (float): H(Y | X).
    """
    # 1. Extraer las realizaciones de X e Y
    X = realizaciones[:, 0]
    Y = realizaciones[:, 1]
    N = len(X)

    # 2. Obtener el alfabeto (valores únicos) de X e Y
    alfabeto_x = np.unique(X)
    alfabeto_y = np.unique(Y)
    Mx = len(alfabeto_x)
    My = len(alfabeto_y)

    # 3. Crear una matriz de frecuencia conjunta
    frecuencia_conjunta = np.zeros((Mx, My), dtype=float)

    # 4. Calcular las frecuencias conjuntas
    for i, x_val in enumerate(alfabeto_x):
        for j, y_val in enumerate(alfabeto_y):
            frecuencia_conjunta[i, j] = np.sum((X == x_val) & (Y == y_val))

    # Probabilidad conjunta
    pXY = frecuencia_conjunta / N

    # 5. Calcular las frecuencias/probabilidades marginales de Y y X
    pX = np.sum(pXY, axis=1)  # tamaño (Mx,)
    pY = np.sum(pXY, axis=0)  # tamaño (My,)

    # 6. Calcular las probabilidades condicionales P(X | Y) y P(Y | X)
    # P(X|Y=j) = pXY[:,j] / pY[j]
    probabilidad_X_dado_Y = np.zeros((Mx, My), dtype=float)
    for j in range(My):
        if pY[j] > 0:
            probabilidad_X_dado_Y[:, j] = pXY[:, j] / pY[j]

    # P(Y|X=i) = pXY[i,:] / pX[i]
    probabilidad_Y_dado_X = np.zeros((Mx, My), dtype=float)
    for i in range(Mx):
        if pX[i] > 0:
            probabilidad_Y_dado_X[i, :] = pXY[i, :] / pX[i]

    # 7. Calcular las entropías condicionales H(X | Y) y H(Y | X)
    # H(X|Y) = sum_y p(y) * H(X|Y=y)
    entropia_X_dado_Y = 0.0
    for j in range(My):
        if pY[j] > 0:
            p = probabilidad_X_dado_Y[:, j]
            p = p[p > 0]
            entropia_X_dado_Y += pY[j] * (-np.sum(p * np.log2(p)))

    # H(Y|X) = sum_x p(x) * H(Y|X=x)
    entropia_Y_dado_X = 0.0
    for i in range(Mx):
        if pX[i] > 0:
            p = probabilidad_Y_dado_X[i, :]
            p = p[p > 0]
            entropia_Y_dado_X += pX[i] * (-np.sum(p * np.log2(p)))

    return (alfabeto_x, alfabeto_y,
            probabilidad_X_dado_Y, probabilidad_Y_dado_X,
            entropia_X_dado_Y, entropia_Y_dado_X)

# Calcular entropías condicionales
_, _, _, _, H_X_dado_Y, H_Y_dado_X = calculo_entropia_condicional(XY)

print(f"H(X | Y) = {H_X_dado_Y:.4f} bits")
print(f"H(Y | X) = {H_Y_dado_X:.4f} bits")


H(X | Y) = 1.5845 bits
H(Y | X) = 1.5844 bits


## 4. Cálculo información mutua

In [ ]:
def calculo_informacion_mutua(realizaciones):
    """
    Calcula la información mutua entre dos variables aleatorias discretas.

    Parámetros:
        realizaciones (array): Matriz N x 2 con las realizaciones de dos variables aleatorias discretas.

    Retorno:
        informacionMutua (float): Escalar que representa la información mutua entre las dos variables.
    """
    _, _, _, entropiaX, entropiaY, entropiaConjunta = calculo_entropia_conjunta(realizaciones)
    informacionMutua = entropiaX + entropiaY - entropiaConjunta
    return informacionMutua

# Calcular la información mutua
informacionMutua = calculo_informacion_mutua(XY)
informacionMutua

np.float64(0.4153283012304363)

## 5. Capacidad canal

In [ ]:
##Antes de calcular la capacidad del canal (si los datos son raros)
import numpy as np

def estimar_canal_desde_XY(XY):
    """
    Estima P(Y|X) a partir de pares (X,Y) observados.
    Devuelve alfabetos y matriz P con filas=X y columnas=Y.
    """
    XY = np.array(XY)
    X_in = XY[:, 0].ravel()
    Y_out = XY[:, 1].ravel()

    alf_x = np.unique(X_in)
    alf_y = np.unique(Y_out)

    Mx, My = len(alf_x), len(alf_y)

    # Conteos conjuntos N(x,y)
    Nxy = np.zeros((Mx, My), dtype=float)
    for i, xv in enumerate(alf_x):
        mask_x = (X_in == xv)
        for j, yv in enumerate(alf_y):
            Nxy[i, j] = np.sum(mask_x & (Y_out == yv))

    # Conteos marginales N(x)
    Nx = Nxy.sum(axis=1, keepdims=True)

    # P(Y|X)=N(x,y)/N(x)
    P = np.divide(Nxy, Nx, out=np.zeros_like(Nxy), where=(Nx > 0))

    return alf_x, alf_y, P

XY_real = data_XY['XY']   # el tuyo
alf_x, alf_y, P_est = estimar_canal_desde_XY(XY_real)





In [ ]:
#OJO solo vale para matrices 4*4
def capacidad_canal_44(P):
    """
    Calcula la capacidad de un canal discreto 4x4 y la distribución de entrada óptima (búsqueda en rejilla).

    Parámetros:
        P (2D array): Matriz 4x4 con P(Y|X). Filas: X, Columnas: Y.

    Retornos:
        capacidad_canal (float): Capacidad aproximada (por rejilla).
        pX_optima (array): Vector p(X) que maximiza I(X;Y).
    """
    P = np.array(P, dtype=float)
    assert P.shape == (4, 4), "P debe ser 4x4"

    capacidad_canal = -np.inf
    pX_optima = np.zeros(4)
    paso = 0.05
    eps = 1e-15

    # Recorrer todas las combinaciones posibles de pX (con paso de 0.05)
    for p1 in np.arange(0, 1 + paso, paso):
        for p2 in np.arange(0, 1 - p1 + paso, paso):
            for p3 in np.arange(0, 1 - p1 - p2 + paso, paso):
                p4 = 1 - p1 - p2 - p3  # La última probabilidad depende de las otras tres
                if p4 < -1e-12 or p4 > 1 + 1e-12:
                    continue  # combinaciones no válidas por error numérico

                # Forzar a [0,1] por pequeños errores flotantes
                p4 = min(max(p4, 0.0), 1.0)

                # Vector de probabilidades de entrada
                pX = np.array([p1, p2, p3, p4], dtype=float)

                # Calcular la salida p(Y) = sum_x P(Y|X=x) * p(X=x)
                # Si P es filas=X, cols=Y: pY[j] = sum_i pX[i] * P[i,j]
                pY = pX @ P  # (4,) @ (4,4) -> (4,)

                # Calcular la entropía de salida H(Y)
                pY_pos = pY[pY > 0]
                H_Y = -np.sum(pY_pos * np.log2(pY_pos + eps))

                # Calcular la entropía condicional H(Y|X) = sum_x p(x) H(Y|X=x)
                H_Y_dado_X = 0.0
                for i in range(4):
                    if pX[i] > 0:
                        row = P[i, :]
                        row_pos = row[row > 0]
                        H_row = -np.sum(row_pos * np.log2(row_pos + eps))
                        H_Y_dado_X += pX[i] * H_row

                # Calcular la información mutua I(X;Y) = H(Y) - H(Y|X)
                I_XY = H_Y - H_Y_dado_X

                # Actualizar la capacidad de canal y pX óptima si se encuentra un valor mayor
                if I_XY > capacidad_canal:
                    capacidad_canal = I_XY
                    pX_optima = pX.copy()

    return capacidad_canal, pX_optima



# Calcular la capacidad del canal y el vector de entrada óptimo
capacidad_canal, pX_optima = capacidad_canal_44(P_est)
capacidad_canal, pX_optima

(np.float64(0.41539549847121116), array([0.25, 0.25, 0.25, 0.25]))